In [37]:
import random

class EventDrivenSimulation:
    def __init__(self, lambda_arrival_regular, lambda_arrival_priority, mu_service, simulation_time):
        self.lambda_arrival_regular = lambda_arrival_regular
        self.lambda_arrival_priority = lambda_arrival_priority
        self.mu_service = mu_service
        self.simulation_time = simulation_time
        self.current_time = 0
        self.events = []  # Lista zdarzeń
        self.history = []  # Historia zdarzeń
        self.regular_queue = []  # Kolejka bez priorytetu
        self.priority_queue = []  # Kolejka z priorytetem
        self.is_serving = False
        self.df_pq = pd.DataFrame(columns=['arrival_time', 'service_time', 'queue_time', 'total_time', 'end_time', 'priority'])
        self.df_q = pd.DataFrame(columns=['arrival_time', 'service_time', 'queue_time', 'total_time', 'end_time', 'priority'])
        self.df_q_num = 0
        self.df_pq_num = 0
        
    def initialize(self):
        # Inicjalizacja pierwszych zdarzeń przybycia
        self.schedule_event('A', random.expovariate(self.lambda_arrival_regular))
        self.schedule_event('B', random.expovariate(self.lambda_arrival_priority))
    
            
    def schedule_event(self, event_type, event_time):
        self.events.append((self.current_time + event_time, event_type))
        self.events.sort()  # Sortowanie zdarzeń po czasie
        
    def run(self):
        self.initialize()
        
        while self.current_time < self.simulation_time and self.events:
            print(self.events)
            # Pobieranie następnego zdarzenia
            self.current_time, event_type = self.events.pop(0)
            self.history.append((self.current_time, event_type))
            # A - przybycie zwykłego klienta
            # B - przybycie klienta z priorytetem
            # C - obsługa klienta w okienku
            # D - przejście klienta do okienka
            # E - przejście klienta z priorytetem do okienka
            if event_type == 'A':
                self.add_record(0)
                self.handle_arrival(False)
            elif event_type == 'B':
                self.add_record(1)
                self.handle_arrival(True)
            elif event_type == 'C':
                self.handle_service()
            elif event_type == 'D':
                self.handle_decide_regular_queue()
            elif event_type == 'E':
                self.handle_decide_priority_queue()
#             print(self.events)
#             print("\n",self.regular_queue)
#             print("\n",self.priority_queue)
                
        return self.history
    
    def handle_arrival(self, is_priority): # jeśli przybył klient w zależności od klasy wylosuj wydarzenie A lub B w zależności od priorytetu
        if is_priority:
            self.priority_queue.append(self.current_time)
            self.schedule_event('B', random.expovariate(self.lambda_arrival_priority))
        else:
            self.regular_queue.append(self.current_time)
            self.schedule_event('A', random.expovariate(self.lambda_arrival_regular))
            
        # Planowanie decyzji o obsłudze klienta, jeśli nie jest zajęty
        if not self.is_serving:
            if self.priority_queue:
                self.schedule_event('E', 0)  # Decyzja o obsłudze priorytetowego klienta
            elif self.regular_queue:
                self.schedule_event('D', 0)  # Decyzja o obsłudze zwykłego klienta
    
    def handle_service(self):
        self.is_serving = False
        
        # Planowanie decyzji o obsłudze klienta, jeśli kolejka nie jest pusta gdzie decydujemy, który klient jest obsługiwany prio czy nie
        if self.priority_queue:
            self.schedule_event('E', 0)  # Decyzja o obsłudze priorytetowego klienta
        elif self.regular_queue:
            self.schedule_event('D', 0)  # Decyzja o obsłudze zwykłego klienta
    
    def handle_decide_regular_queue(self): # obsługa eventu zajmującego się zwykłą kolejką gdzie losowany jest czas obsługi
        # i dodawany jest event związany z obsługą klienta (C)
        if self.regular_queue:
            self.is_serving = True
            self.regular_queue.pop(0)
            service_time = random.expovariate(self.mu_service)
            self.add_time(0,service_time )
            self.schedule_event('C', service_time)
    
    def handle_decide_priority_queue(self): # obsługa eventu zajmującego się priorytetową kolejką gdzie losowany jest czas obsługi
        # i dodawany jest event związany z obsługą klienta (C)
        if self.priority_queue:
            self.is_serving = True
            self.priority_queue.pop(0)
            service_time = random.expovariate(self.mu_service)
            self.add_time(1,service_time )
            self.schedule_event('C', service_time)
            
    def add_record(self, priority):
        if priority:
            self.df_pq.loc[len(self.df_pq)] = [self.current_time, None, None, None,None, priority]
        else:
            self.df_q.loc[len(self.df_q)] = [self.current_time, None, None, None,None, priority]
        
    def add_time(self, priority, service_time):
        end_time = self.current_time + service_time
        if priority:
            total_time = end_time - self.df_pq.iloc[self.df_pq_num,0]
            q_time = total_time - service_time
            self.df_pq.loc[self.df_pq_num,['service_time', 'queue_time', 'end_time', 'total_time']]= [service_time, q_time, self.current_time+service_time ,total_time]
            self.df_pq_num+=1
        else:
            total_time = end_time - self.df_q.iloc[self.df_q_num,0]
            q_time = total_time - service_time
            self.df_q.loc[self.df_q_num,['service_time', 'queue_time', 'end_time', 'total_time']]= [service_time, q_time, self.current_time+service_time,total_time]
            self.df_q_num+=1

# Parametry symulacji
lambda_arrival_regular = 10  # średnia liczba przybyć klientów zwykłych na jednostkę czasu
lambda_arrival_priority = 3  # średnia liczba przybyć klientów z priorytetem na jednostkę czasu (częstsze przybycia)
mu_service = 15.0      # średnia liczba obsłużonych klientów na jednostkę czasu
simulation_time = 20.0  # całkowity czas symulacji

# Tworzenie i uruchomienie symulacji
simulation = EventDrivenSimulation(lambda_arrival_regular, lambda_arrival_priority, mu_service, simulation_time)
history = simulation.run()

# Wyświetlanie historii zdarzeń
for event in history:
    print(event)

[(0.015149057795165297, 'A'), (0.6762843894364697, 'B')]
[(0.015149057795165297, 'D'), (0.06774443306394715, 'A'), (0.6762843894364697, 'B')]
[(0.040720925088221435, 'C'), (0.06774443306394715, 'A'), (0.6762843894364697, 'B')]
[(0.06774443306394715, 'A'), (0.6762843894364697, 'B')]
[(0.06774443306394715, 'D'), (0.13171650853851202, 'A'), (0.6762843894364697, 'B')]
[(0.10900416313045291, 'C'), (0.13171650853851202, 'A'), (0.6762843894364697, 'B')]
[(0.13171650853851202, 'A'), (0.6762843894364697, 'B')]
[(0.13171650853851202, 'D'), (0.4245837619802617, 'A'), (0.6762843894364697, 'B')]
[(0.16559547468533628, 'C'), (0.4245837619802617, 'A'), (0.6762843894364697, 'B')]
[(0.4245837619802617, 'A'), (0.6762843894364697, 'B')]
[(0.4245837619802617, 'D'), (0.4961240624646568, 'A'), (0.6762843894364697, 'B')]
[(0.4961240624646568, 'A'), (0.5582020423459597, 'C'), (0.6762843894364697, 'B')]
[(0.5582020423459597, 'C'), (0.5940771101410224, 'A'), (0.6762843894364697, 'B')]
[(0.5582020423459597, 'D')

In [39]:
simulation.df_pq

,arrival_time,service_time,queue_time,total_time,end_time,priority
0,0.676284,0.008237,1.727754e-01,0.181012,0.857297,1.0
1,0.971108,0.099197,9.258055e-02,0.191778,1.162886,1.0
2,1.517704,0.058715,1.810894e-02,0.076824,1.594528,1.0
3,1.805287,0.159550,1.592744e-01,0.318824,2.124111,1.0
4,1.872231,0.018125,2.518805e-01,0.270005,2.142236,1.0
5,1.947728,0.220321,1.945083e-01,0.414829,2.362557,1.0
6,2.251920,0.004346,1.106366e-01,0.114983,2.366903,1.0
7,2.333037,0.004759,3.386664e-02,0.038626,2.371663,1.0
8,2.867730,0.025839,1.639097e-01,0.189749,3.057479,1.0
9,3.127129,0.068383,1.380282e-01,0.206411,3.333540,1.0
